## Read Data

In [1]:
import dask.dataframe as dd
import pandas as pd
fp = "../../data/incident_event_log_02.csv"
df = pd.read_csv(fp)

/var/folders/fb/t_m5qpcj6qq85rvkh73vbxh40000gn/T/ipykernel_7631/3477932692.py:4: DtypeWarning: Columns (19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fp)


## Select Columns Needed

In [2]:
SELECT_COLS = ['number', 'sys_created_at', 'closed_at', 'assignment_group']
closed_tickets = df.incident_state == "Closed"
df_closed_tickets = df[closed_tickets][SELECT_COLS].copy()
del df
df_closed_tickets = df_closed_tickets.reset_index(drop=True)

## Inspect Null Information in Dataset

In [3]:
df_closed_tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24985 entries, 0 to 24984
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   number            24985 non-null  object
 1   sys_created_at    13466 non-null  object
 2   closed_at         24985 non-null  object
 3   assignment_group  22816 non-null  object
dtypes: object(4)
memory usage: 780.9+ KB


## Noise Filter #1 definition

In [4]:
def noise_filter_1(row):
    creation_date = row['sys_created_at']
    valid_creation_date = True
    valid_closed_date = True
    cd = cls_dt = None
    if pd.isna(creation_date):
        valid_creation_date = False
    else:
        try:
            creation_date = pd.to_datetime(creation_date, infer_datetime_format=True)
        except:
            valid_creation_date = False
    
            
    closed_date = row["closed_at"]
    
    if pd.isna(closed_date):
        valid_closed_date = False
    else:
        try:
            cls_dt = pd.to_datetime(closed_date)

        except:
            valid_closed_date = False
    
    
    clean_row = valid_creation_date & valid_closed_date
    

    return valid_creation_date


## Apply Noise Filter #1

In [5]:
df_closed_tickets = df_closed_tickets[df_closed_tickets.apply(noise_filter_1, axis=1)]

## Define Noise Filter #2

In [6]:
dict_types = {"number": 'str', "sys_created_at": 'datetime64[ns]',
              "closed_at": "datetime64[ns]" , "assignment_group": 'str'}
df_closed_tickets = df_closed_tickets.astype(dict_types)
valid_closing_dates = df_closed_tickets["closed_at"] > df_closed_tickets["sys_created_at"]
q2_2016 = (df_closed_tickets.closed_at.dt.quarter == 2) & \
        (df_closed_tickets.closed_at.dt.year == 2016)
in_range_good_tickets = q2_2016 & valid_closing_dates

## Apply Noise Filter #2

In [7]:

df_closed_tickets = df_closed_tickets[in_range_good_tickets].reset_index(drop=True)

## Write Report Data

In [8]:
fp = "../../data/q2_2016_ticket_resolution_data.csv"
df_closed_tickets.to_csv(fp, index=False)

In [9]:
dtypes_meta = {"attribute": [], "type": []}
for k, v in dict_types.items():
    dtypes_meta["attribute"].append(k)
    dtypes_meta["type"].append(v)
    
df_dtypes = pd.DataFrame.from_dict(dtypes_meta)
fp_types = "../../data/ticket_resolution_dtypes.csv"
df_dtypes.to_csv(fp_types, index=False)